In [ ]:
# !pip install fuzzywuzzy

In [2]:
import pandas as pd
from pprint import pprint
from IPython.display import display
from fuzzywuzzy import fuzz

c:\Users\soiko\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
research_gate = pd.read_csv('data/info/info_full.csv')
print(research_gate.shape)
print(research_gate.columns)
research_gate.head(n=1)

(18192, 12)
Index(['id', 'parent_id', 'name', 'type', 'published_date', 'references_count',
       'research_interest_score', 'citations_count', 'recommendations_count',
       'reads_count', 'paper_link', 'download_link'],
      dtype='object')


,id,parent_id,name,type,published_date,references_count,research_interest_score,citations_count,recommendations_count,reads_count,paper_link,download_link
0,8bde6cb8-296d-4593-883f-b768103e1e69,NaN,PositionRank: An Unsupervised Approach to Keyp...,Article,2017-08-01T00:00:00.000Z,40,12.2,10,1,621,https://www.researchgate.net/publication/31859...,https://www.researchgate.net/profile/Corina-Fl...


In [4]:
semantic = pd.read_csv("data/info_ss/info_full_ss.csv")
print(semantic.shape)
print(semantic.columns)
semantic.head(n=1)

(24750, 17)
Index(['uuid', 'paper_id', 'parent_id', 'title', 'reference_count',
       'citation_count', 'influential_citation_count', 'published_date',
       'paper_type', 'venue', 'isOpenAccess', 'abstract', 'paper_link', 'doi',
       'download_link', 'tldl', 'embedding'],
      dtype='object')


,uuid,paper_id,parent_id,title,reference_count,citation_count,influential_citation_count,published_date,paper_type,venue,isOpenAccess,abstract,paper_link,doi,download_link,tldl,embedding
0,8bde6cb8-296d-4593-883f-b768103e1e69,b0d555a9ea67285fccd2ef8d887907bcc811f67a,NaN,PositionRank: An Unsupervised Approach to Keyp...,43,176,26,2017.0,Article,ACL,True,The large and growing amounts of online schola...,https://www.semanticscholar.org/paper/b0d555a9...,https://doi.org/10.18653/v1%2FP17-1102,https://www.aclweb.org/anthology/P17-1102.pdf;...,tldr@v2.0.0:An unsupervised model for keyphras...,"specter@v0.1.1:-3.3292479515075684,-1.61703908..."


In [5]:
research_gate.iloc[0]['name']

'PositionRank: An Unsupervised Approach to Keyphrase Extraction from Scholarly Documents'

In [6]:
paper = research_gate.iloc[0]['name']
paper_found = semantic.loc[semantic['title'].str.contains(paper[:20],case=False)].title[0]
paper_found

'PositionRank: An Unsupervised Approach to Keyphrase Extraction from Scholarly Documents'

In [7]:
ratio = fuzz.partial_ratio(paper.lower(),paper_found.lower())
ratio

100

In [8]:
count = 0
for i, paper_tuple in enumerate(semantic.iterrows()):
	paper_pdSeries = paper_tuple[1]
	paper_title_semantic = paper_pdSeries['title']
	paper_uuid_semantic = paper_pdSeries['uuid']
	paper_download_url_semantic = paper_pdSeries['download_link']
	# paper_type = paper_pdSeries['type']
	# print(paper_title_semantic)
	try:
		paper_found = research_gate.loc[research_gate['name'].str.contains(
			paper_title_semantic, case=False)].to_dict('records')
		found_paper_1 = None
		if paper_found:
			found_paper_1 = paper_found[0]
		else:
			# print(f"NotFound: {paper_title_semantic}")
			semantic.loc[semantic['uuid'] ==
						paper_uuid_semantic, 'paper_type'] = "Unknown"
			
			continue
		# print(type(found_paper_1))
		# print(found_paper_1)
		found_paper_title = found_paper_1['name']
		# print(found_paper_title)
		found_paper_type = found_paper_1['type']
		# print(found_paper_type)
		found_paper_download_link = found_paper_1['download_link']
		
		semantic.loc[semantic['uuid'] == paper_uuid_semantic, 'paper_type'] = found_paper_type
		semantic.loc[semantic['uuid'] == paper_uuid_semantic,
				'download_link'] = f"{paper_download_url_semantic};{found_paper_download_link}"
		count += 1
	except:
		semantic.loc[semantic['uuid'] ==
				paper_uuid_semantic, 'paper_type'] = "Unknown"


C:\Users\soiko\AppData\Local\Temp/ipykernel_16132/2858471905.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  paper_found = research_gate.loc[research_gate['name'].str.contains(


In [9]:
semantic.head(n=3)

,uuid,paper_id,parent_id,title,reference_count,citation_count,influential_citation_count,published_date,paper_type,venue,isOpenAccess,abstract,paper_link,doi,download_link,tldl,embedding
0,8bde6cb8-296d-4593-883f-b768103e1e69,b0d555a9ea67285fccd2ef8d887907bcc811f67a,NaN,PositionRank: An Unsupervised Approach to Keyp...,43,176,26,2017.0,Article,ACL,True,The large and growing amounts of online schola...,https://www.semanticscholar.org/paper/b0d555a9...,https://doi.org/10.18653/v1%2FP17-1102,https://www.aclweb.org/anthology/P17-1102.pdf;...,tldr@v2.0.0:An unsupervised model for keyphras...,"specter@v0.1.1:-3.3292479515075684,-1.61703908..."
1,ade6eb61-946a-49f3-835d-74804cb9de3e,03589e1917debe6df148cac8963fd008e4140237,8bde6cb8-296d-4593-883f-b768103e1e69,SemEval-2010 Task 5 : Automatic Keyphrase Extr...,29,352,71,2010.0,Unknown,*SEMEVAL,False,This paper describes Task 5 of the Workshop on...,https://www.semanticscholar.org/paper/03589e19...,NaN,https://www.aclweb.org/anthology/S10-1004.pdf;...,tldr@v2.0.0:The participating systems were eva...,"specter@v0.1.1:-5.0251593589782715,-0.76686722..."
2,be2af112-6c57-4598-9813-d02fff71055b,05dec9ff0f48f45b09245ba0354748961f18ff77,8bde6cb8-296d-4593-883f-b768103e1e69,Citation-Enhanced Keyphrase Extraction from Re...,61,100,10,2014.0,Conference Paper,EMNLP,True,Given the large amounts of online textual docu...,https://www.semanticscholar.org/paper/05dec9ff...,https://doi.org/10.3115/v1%2FD14-1150,https://www.aclweb.org/anthology/D14-1150.pdf;...,tldr@v2.0.0:Novel features based on citation n...,"specter@v0.1.1:-4.417524337768555,-1.956189870..."


In [62]:
semantic.iloc[1001]['paper_type']

'Chapter'

In [10]:
semantic.to_csv('data/info_ss/info_full_ss_new.csv', index=False)


In [11]:
semantic_new = pd.read_csv('data/info_ss/info_full_ss_new.csv')
semantic_new['paper_type'].value_counts()

Unknown              10910
Article               8851
Conference Paper      4352
Book                   429
Chapter                182
Literature Review       26
Name: paper_type, dtype: int64

In [9]:
semantic_new['paper_id'].nunique()

9644

In [22]:
semantic_new[semantic_new['published_date'].isnull()]['paper_id'].unique()

array(['d835c665aa0a14f4269d6ad9c8c2eec0dffc1f9c',
       '12e6f6b2da067b61cf692278b8d9933452eee252',
       '8d0c45a7b66fa1806ff9ecf4f54f9fbc2c104c73',
       'c768610307a21e565af79153371d2e4c4147dd09',
       '6f1527b380baa0af59964f562dfb2ad71650739a',
       '3da8684050df72c07731c030f7669a920ccfffde',
       '8c1ec40d3b4846beaf7fcdb0b99208469a3d44d4',
       'f1f4386524be3ed96caaf05f661aacb94db1e566',
       '1bddad4dc0dfa8efa402aa5d18c29304a5760f12',
       '899a09aa8a2d76a2eea6a2632523deb1e87cec22',
       'bd2e048c676ad778351bd7d7660240a978422117',
       'ed3b54d37c91ccaf7b059e217d1c91deab042e71',
       '50bc1be59e7dee5ee5b26710fd862592498fa0aa',
       '09fefeaa20be1fb689bd860faa0371ca51eff626',
       'da09c4ee72e67d23967adab0466baffc7b1a0257',
       'c56c37eebfe2acb72ccfc63cb0d696b81a1808e3',
       '8829f85132c8edaee975e281d8309aea578fe22b',
       'f07d6ba67f189eeb3009773c354bb736d0363e30',
       'b48355429f2457dddbb8b70ad7e1116125286ecf',
       '003afdf870ed4f219873e25